In [1]:
import logging
import re
import ssl
import time
from datetime import date
from functools import reduce

import numpy as np
import pandas as pd
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from slack_sdk.web.slack_response import SlackResponse
from sqlalchemy import MetaData, Table
from sqlalchemy.sql import and_, case, func, or_, select, text

from pax_achievements import (
    cadre,
    el_presidente,
    hammer_not_nail,
    hdtf,
    leader_of_men,
    nation_sql,
    posts,
    region_sql,
    six_pack,
    the_boss,
    the_monk,
    the_priest,
)
from utils import mysql_connection, slack_client


In [2]:
logging.basicConfig(format="%(asctime)s [%(levelname)s]:%(message)s",
                        level=logging.INFO,
                        datefmt="%Y-%m-%d %H:%M:%S")

In [3]:
year = date.today().year
engine = mysql_connection()
metadata = MetaData()
%time metadata.reflect(engine, schema="weaselbot")

CPU times: user 73.1 ms, sys: 14.2 ms, total: 87.3 ms
Wall time: 2.76 s


In [4]:
%time df_regions = pd.read_sql(region_sql(metadata), engine)
%time nation_df = pd.read_sql(nation_sql(metadata, year), engine, parse_dates="date")

CPU times: user 12 ms, sys: 4.05 ms, total: 16 ms
Wall time: 142 ms
CPU times: user 1.2 s, sys: 446 ms, total: 1.65 s
Wall time: 1min 2s


In [44]:
bb_filter = nation_df.backblast.str.lower().str[:100].str.contains(r"q.{0,1}source|q{0,1}[1-9]\.[0-9]\s|ruck")
ao_filter = nation_df.ao.str.contains(r"q.{0,1}source|ruck")

In [6]:
############# Q Source ##############
priest_df = the_priest(nation_df, bb_filter, ao_filter)
monk_df = the_monk(nation_df, bb_filter, ao_filter)
############### END #################

In [7]:
############ ALL ELSE ###############
leader_of_men_df = leader_of_men(nation_df, bb_filter, ao_filter)
boss_df = the_boss(nation_df, bb_filter, ao_filter)
hammer_not_nail_df = hammer_not_nail(nation_df, bb_filter, ao_filter)
cadre_df = cadre(nation_df, bb_filter, ao_filter)
el_presidente_df = el_presidente(nation_df, bb_filter, ao_filter)

ss = []
s = posts(nation_df, bb_filter, ao_filter)
for val in [25, 50, 100, 150, 200]:
    ss.append(s[s >= val])
el_quatro, golden_boy, centurian, karate_kid, crazy_person = ss
six_pack_df = six_pack(nation_df, bb_filter, ao_filter)
hdtf_df = hdtf(nation_df, bb_filter, ao_filter)

In [8]:
df_regions.paxminer_schema.unique()

array(['f3alliance', 'f3waco', 'f3naperville', 'f3waxhaw', 'f3columbia',
       'f3muletown', 'f3pugetsound', 'f3jeffco', 'f3stlmeramec',
       'f3southcary', 'f3glacier', 'f3ttown', 'f3washmo', 'f3expedition',
       'f3central_il', 'f3southsound', 'f3thecapital', 'f3nwarkansas',
       'f3thespark', 'f3gwinnett', 'f3stl', 'f3orlando', 'f3westcobb',
       'f3marshall', 'f3roanoke', 'f3albany', 'f3suncoast', 'f3austin',
       'f3hillcountry', 'f3blueridge', 'f3portland', 'f3northlake',
       'f3cherokee', 'f3denver', 'f3florence', 'f3fortcollins',
       'f3thejunction', 'f3wichita', 'f3noho', 'f3knoxville', 'f3thefort',
       'f3houston', 'f3kcmo', 'f3borderlands', 'f3westindy', 'f3soco',
       'f3huntsville', 'f3flomo', 'f3albuquerque', 'f3spiritofstl',
       'f3ne_wake', 'f3stcharles', 'f3thechuck', 'f3chicago',
       'f3se_michigan', 'f3brentwood', 'f3sclt', 'f3twincities',
       'f3memphis', 'f3noco', 'f3tornadoalley'], dtype=object)

In [36]:
for r in df_regions.itertuples(index=False):
    if r.paxminer_schema == "f3gwinnett":
        row = r

In [ ]:
# client = slack_client(row.slack_token) # no need for this yet. Here for completeness

In [37]:
idx = pd.IndexSlice
try:
    records = priest_df[idx[:, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=1, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in the_priest")

2024-01-29 04:46:34 [ERROR]:f3gwinnett not in the_priest


In [38]:
try:
    records = monk_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=2, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in the_monk")

2024-01-29 04:46:35 [ERROR]:f3gwinnett not in the_monk


In [45]:
try:
    records = leader_of_men_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=3, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in leader_of_men")

In [46]:
data

[{'month': 1,
  'slack_user_id': 'U0201J33A4T',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U0202J8T8LD',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U020EMVJ6M7',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U029PFFFX6U',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U03D4QFRLCB',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U03D7F9MB8C',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U03UKPWF61G',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)},
 {'month': 1,
  'slack_user_id': 'U05RYV3SNKD',
  'achievement_id': 3,
  'date_awarded': datetime.date(2024, 1, 29)}]

In [47]:
nation_df.query("slack_user_id == 'U03UKPWF61G' and not (@bb_filter or @ao_filter) and (q_flag == 1)")

,email,user_name,slack_user_id,ao_id,ao,date,year_num,month_num,week_num,day_num,q_flag,backblast,home_region
18361,jonathan@theahc.net,Chef (18/2of50lbs),U03UKPWF61G,2875,ao-coliseum,2024-01-10,2024,1,1,10,1,Backblast: Alternating Q Wednesday \nDate: 01/...,f3gwinnett
18364,jonathan@theahc.net,Chef (18/2of50lbs),U03UKPWF61G,2919,ao-wolf-den,2024-01-13,2024,1,1,13,1,Backblast: Easy Breezy Saturday\nDate: 1/13/24...,f3gwinnett


In [17]:
try:
    records = boss_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=4, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in the_boss")

In [18]:
data

[{'month': 1,
  'slack_user_id': 'U04KKCDRRAA',
  'achievement_id': 4,
  'date_awarded': datetime.date(2024, 1, 29)}]

In [19]:
try:
    records = hammer_not_nail_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=5, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in be_the_hammer_not_the_nail")

2024-01-29 04:37:11 [ERROR]:f3alliance not in be_the_hammer_not_the_nail


In [20]:
try:
    records = cadre_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=6, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in cadre")

2024-01-29 04:37:13 [ERROR]:f3alliance not in cadre


In [21]:
try:
    records = el_presidente_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=7, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in el_presidente")

2024-01-29 04:37:14 [ERROR]:f3alliance not in el_presidente


In [22]:
try:
    records = el_quatro[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=8, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in el_quatro")

2024-01-29 04:37:14 [ERROR]:f3alliance not in el_quatro


In [23]:
try:
    records = golden_boy[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=9, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in golden_boy")

2024-01-29 04:37:15 [ERROR]:f3alliance not in golden_boy


In [24]:
try:
    records = centurian[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=10, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in centurian")

2024-01-29 04:37:18 [ERROR]:f3alliance not in centurian


In [25]:
try:
    records = karate_kid[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=11, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in karate_kid")

2024-01-29 04:37:19 [ERROR]:f3alliance not in karate_kid


In [26]:
try:
    records = crazy_person[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=12, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in crazy_person")

2024-01-29 04:37:19 [ERROR]:f3alliance not in crazy_person


In [27]:
try:
    records = six_pack_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao_id", axis=1).assign(achievement_id=13, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in six_pack")

In [35]:
try:
    records = hdtf_df[idx[:, :, row.paxminer_schema]]
    data = records.reset_index().drop("ao", axis=1).assign(achievement_id=14, date_awarded=date.today()).to_dict("records")
except KeyError as e:
    logging.error(f"{row.paxminer_schema} not in holding_down_the_fort")

2024-01-29 04:45:07 [ERROR]:f3alliance not in holding_down_the_fort


In [ ]:
# under current logic, this man received 6-pack. But he hasn't earned it because 2 of his beatdowns were Q Source. New
# logic catches it.

nation_df.query("slack_user_id == 'U0328DW47MH' and '2024-01-21' <= date <= '2024-01-27' and ao != 'ao_q_source'")